In [ ]:
import cv2
import numpy as np

In [ ]:
# Open the default USB camera (camera index 0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

In [ ]:
while True:
    # 2. 从摄像头读取一帧图像
    ret, frame = cap.read()

    # 检查是否成功读取帧
    if not ret:
        print("Error: Could not read frame.")
        break

    bottle_rect = get_bottle_obb_edge(frame)
    
    # 3. 绘制结果
    if bottle_rect:
        box = cv2.boxPoints(bottle_rect)
        box = np.int0(box)
        cv2.drawContours(img, [box], 0, (0, 0, 255), 2)  # 红色矩形
        cv2.imshow('Bottle Bounding Box (Edge)', img)
        cv2.waitKey(1)
        
    else:
        print('未找到瓶子')
        cv2.destroyAllWindows()
#     # 3. 转换为灰度图
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # 4. 二值化 (使用 Otsu's 二值化)
#     thresh, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

#     # 5. 轮廓提取
#     contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # 6. 绘制轮廓 (在原始彩色帧上绘制)
#     frame_with_contours = frame.copy()
#     cv2.drawContours(frame_with_contours, contours, -1, (0, 0, 255), 2)

#     # 7. 显示结果
#     cv2.imshow('Original Camera Feed', frame)
#     cv2.imshow('Binary Image', binary)
#     cv2.imshow('Camera Feed with Contours', frame_with_contours)

#     # 8. 按下 'q' 键退出循环
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # 9. 释放摄像头资源并关闭窗口
# cap.release()
# cv2.destroyAllWindows()

# 你可以在循环中添加对轮廓的处理代码，例如计算面积、最小外接矩形等。
# 示例：
# for contour in contours:
#     area = cv2.contourArea(contour)
#     if area > some_threshold:
#         rect = cv2.minAreaRect(contour)
#         box = cv2.boxPoints(rect)
#         box = np.int0(box)
#         cv2.drawContours(frame_with_contours, [box], 0, (0, 255, 0), 2)

In [ ]:
import cv2
import numpy as np

def get_bottle_obb_edge(image):
    """
    使用边缘检测和轮廓提取获取图像中瓶子的最小外接矩形

    Args:
        image (numpy.ndarray): 输入图像 (BGR格式)

    Returns:
        tuple: 瓶子的最小外接矩形，格式为 ((center_x, center_y), (width, height), angle)，
               如果未找到瓶子，则返回 None
    """
    # 1. 转换为灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. 边缘检测 (Canny)
    #    调整参数以获得更好的边缘效果
    edges = cv2.Canny(gray, 50, 150)  # 尝试不同的阈值

    # 3. 膨胀操作 (可选，连接断开的边缘)
    kernel = np.ones((5, 5), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)

    # 4. 轮廓提取
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 5. 寻找面积最大的轮廓
    if contours:
        bottle_contour = max(contours, key=cv2.contourArea)
        # 6. 计算最小外接矩形
        rect = cv2.minAreaRect(bottle_contour)
        return rect
    else:
        return None



In [3]:
import cv2
import numpy as np
def get_bottle_obb_edge_from_camera(cap):
    """
    使用边缘检测和轮廓提取从摄像头获取瓶子的最小外接矩形

    Args:
        cap (cv2.VideoCapture):  VideoCapture 对象

    Returns:
        tuple: 瓶子的最小外接矩形，格式为 ((center_x, center_y), (width, height), angle)，
               如果未找到瓶子，则返回 None
    """
    # 1. 从摄像头读取一帧图像
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        return None

    # 2. 转换为灰度图
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 3. 边缘检测 (Canny)
    edges = cv2.Canny(gray, 50, 150)  # 尝试不同的阈值

    # 4. 膨胀操作 (可选，连接断开的边缘)
    kernel = np.ones((5, 5), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)

    # 5. 轮廓提取
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 6. 寻找面积最大的轮廓
    if contours:
        bottle_contour = max(contours, key=cv2.contourArea)
        # 7. 计算最小外接矩形
        rect = cv2.minAreaRect(bottle_contour)
        return rect
    else:
        return None
    


In [5]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

while True:
    # 2. 获取瓶子的最小外接矩形
    bottle_rect = get_bottle_obb_edge_from_camera(cap)
    if bottle_rect:
        ret, frame = cap.read() #add this
        box = cv2.boxPoints(bottle_rect)
        box = np.int32(box)
        cv2.drawContours(frame, [box], 0, (0, 0, 255), 2)  # 红色矩形
        cv2.imshow('Bottle Bounding Box (Edge)', frame)
    else:
        ret, frame = cap.read() # add this
        cv2.imshow('Bottle Bounding Box (Edge)', frame) # show the video even if bottle not detected
        print('未找到瓶子')

    # 3. 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 4. 释放摄像头
cap.release()
cv2.destroyAllWindows()

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO  # 导入 YOLO

def get_bottle_obb_yolo_from_camera(cap):
    """
    使用 YOLOv8 检测瓶子，并获取其最小外接矩形

    Args:
        cap (cv2.VideoCapture): VideoCapture 对象

    Returns:
        tuple: 瓶子的最小外接矩形，格式为 ((center_x, center_y), (width, height), angle)，
               如果未找到瓶子，则返回 None
    """
    # 1. 加载 YOLOv8 模型
    model = YOLO('F:/GitHub/Note/毕业设计思路/python_codes/runs/detect/train5/weights/best.pt')

    # 2. 从摄像头读取一帧图像
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        return None

    # 3. 使用 YOLOv8 进行目标检测
    results = model.predict(frame)

    # 4. 获取瓶子的检测结果
    bottle_rect = None
    for result in results:
        for box in result.boxes:
            if result.names[int(box.cls)] == 'bottle':  # 假设 'bottle' 是瓶子的类别名称
                xyxy = box.xyxy[0].cpu().numpy()  # 获取边界框坐标 (xmin, ymin, xmax, ymax)
                center_x = (xyxy[0] + xyxy[2]) / 2
                center_y = (xyxy[1] + xyxy[3]) / 2
                width = xyxy[2] - xyxy[0]
                height = xyxy[3] - xyxy[1]
                angle = 0  # YOLO 输出的是水平矩形

                # 后处理：使用检测到的边界框区域进行更精确的轮廓提取 (可选)
                cropped_img = frame[int(xyxy[1]):int(xyxy[3]), int(xyxy[0]):int(xyxy[2])]
                cropped_gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
                _, cropped_binary = cv2.threshold(cropped_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                cropped_contours, _ = cv2.findContours(cropped_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if cropped_contours:
                    bottle_contour = max(cropped_contours, key=cv2.contourArea)
                    bottle_rect = cv2.minAreaRect(bottle_contour)
                break  # 找到一个瓶子就退出内层循环
        if bottle_rect:
            break  # 找到一个瓶子就退出外层循环
    return bottle_rect



# 1. 打开 USB 摄像头
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

while True:
    # 2. 获取瓶子的最小外接矩形
    bottle_rect = get_bottle_obb_yolo_from_camera(cap)

    # 3. 绘制结果
    ret, frame = cap.read() # 放在这里确保frame被读取
    if bottle_rect:
        box = cv2.boxPoints(bottle_rect)
        box = np.int32(box)
        cv2.drawContours(frame, [box], 0, (0, 0, 255), 2)
        cv2.imshow('Bottle Bounding Box (YOLO)', frame)
    else:
        cv2.imshow('Bottle Bounding Box (YOLO)', frame) # 始终显示摄像头画面
        print('未找到瓶子')

    # 4. 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 5. 释放摄像头
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 bottle, 7.9ms
Speed: 1.3ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 4.7ms
Speed: 0.7ms preprocess, 4.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 4.9ms
Speed: 0.9ms preprocess, 4.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 5.4ms
Speed: 0.8ms preprocess, 5.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 5.1ms
Speed: 0.7ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 4.7ms
Speed: 0.8ms preprocess, 4.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 4.9ms
Speed: 0.9ms preprocess, 4.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 5.6ms
Speed: 0.9ms preprocess, 5.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x